### 교통 소외 지역의 대중교통 접근성 향상을 위해서

* 지역(강원도 춘천시)의 버스 정류장 정보와 보선 노선 운행 정보를 바탕으로
* 정류장별로 어떤 노선(버스 번호)이 몇 개나 다니는지에 대한 수치적 데이터 분석을 진행하고
* 이를 지도 상에 시각화하는 활동을 수행해보자
* 위성 지도 상에 버스 정류장의 위치 정보를 마커로 표시하고 마커의 색은 전체 버스 정류장 노선 정치횟수의 통계적 분석 횟수를 기준으로 표시해보자


In [ ]:
#위성 지도 생성을 위한 프로그래밍 도구(라이브러리) 설치

!pip install leafmap

In [ ]:
#테이블 데이터(표)를 다루기 적절한 판다스 라이브러리와 춘천시 버스 정류장 위치 정보 파일 데이터를 활용해 데이터를 kang_bus에 저장하기

import pandas as pd

pd.set_option('display.max_rows', None) #줄이 안짤리게 최대로 출력하게 하는 코드

kang_bus = pd.read_csv("https://github.com/CarlosQuperman/2025AIEDAP_SOC_NEW/raw/refs/heads/main/%EA%B0%95%EC%9B%90%ED%8A%B9%EB%B3%84%EC%9E%90%EC%B9%98%EB%8F%84%20%EC%B6%98%EC%B2%9C%EC%8B%9C_%EB%B2%84%EC%8A%A4%EC%A0%95%EB%A5%98%EC%9E%A5%20%EC%9C%84%EC%B9%98%EC%A0%95%EB%B3%B4_20250212.csv",encoding='cp949') ### df에 판다스에 있는 도구 중 read_csv를 사용해 선생님이 준비한 강원도 상가(상권) 데이터를 가져와 저장함

display(kang_bus.info())
display(kang_bus.head()) #처음부터 확인할 데이터의 개수 입력하기

In [ ]:
#정류장명이 동일한 문제를 해결하기 위해 정류장명과 관리번호를 통합하여 새로운 속성값 생성
# 그 전에 정류장명에 있는 공백값을 제거하는 데이터 전처리 진행하기

kang_bus['정류장명'] = kang_bus['정류장명'].str.replace(' ', '') #공백제거
kang_bus['관리번호_정류장명'] = kang_bus['관리번호'].astype(str)+kang_bus['정류장명']
display(kang_bus.info())
display(kang_bus.head()) #처음부터 확인할 데이터의 개수 입력하기

In [ ]:
#정류장의 이름과 관리번호와 정류장명을 통합한 값 중 중복되지 않은 값만 출력해서 데이터 확인해보기

display(kang_bus['정류장명'].unique())
print(len(kang_bus['정류장명'].unique()))
print(len(kang_bus['관리번호_정류장명'].unique()))

In [ ]:
#강원특별자치도 춘천시의 버스 운행정보를 kang_route에 저장하고 데이터의 양상 확인해보기
kang_route = pd.read_csv("https://github.com/CarlosQuperman/2025AIEDAP_SOC_NEW/raw/refs/heads/main/%EA%B0%95%EC%9B%90%ED%8A%B9%EB%B3%84%EC%9E%90%EC%B9%98%EB%8F%84%20%EC%B6%98%EC%B2%9C%EC%8B%9C_%EB%B2%84%EC%8A%A4%EC%A0%95%EB%A5%98%EC%9E%A5%20%EB%85%B8%EC%84%A0%EC%A0%95%EB%B3%B4_20250212.csv",encoding='cp949')

display(kang_route.info())
display(kang_route.head()) #처음부터 확인할 데이터의 개수 입력하기

In [ ]:
#정류장 위치정보와 마찬가지로 정류장명에서 공백을 제거하고 관리번호와 정류장명을 병합한 새로운 속성 만들어주기(동일이름 정류장이지만 방향이 다른 정류장을 구별해주기 위해)
kang_route['정류장명'] = kang_route['정류장명'].str.replace(' ', '')
kang_route['관리번호_정류장명'] = kang_route['정류장'].astype(str)+kang_route['정류장명']
display(kang_route.info())
display(kang_route.head())

In [ ]:
#버스 운행 정보가 저장된 kang_route를 처음부터 끝까지 돌면서 버스 정류장별 노선 정차횟수
#버스 정류장별 정차 노선이름(버스 이름 저장하기)
#dict_bus_station : 정류장별 정차 노선 갯수
#dict_bus_station_name 정류장별 정차 노선 이름

import numpy as np
from scipy import stats

#버스 정류장별 노선 정차횟수 저장 딕셔너리
dict_bus_station = {key:0 for key in list(kang_route['관리번호_정류장명'].unique())}

#버스 정류장별 정차 버스 번호 이름 저장 딕셔너리
dict_bus_station_name = {key:set() for key in list(kang_route['관리번호_정류장명'].unique())}


#버스 노선 정보를 한줄씩 처음부터 끝까지 돌면서 관리번호_정류장명에 지나는 노선 횟수를 계산해서 하나씩 증가하고
#그 정류장에 어떤 버스가 서는지를 저장하고 출력함

for i in kang_route.index :
  station_name = kang_route.loc[i,'관리번호_정류장명']
  bus_number = kang_route.loc[i,'노선번호']+'번'
  if station_name in dict_bus_station:
    dict_bus_station[station_name]+=1
    dict_bus_station_name[station_name].add(bus_number)

print(dict_bus_station)
print(dict_bus_station_name)

#노선별 정차횟수를 계산한 후 내용 출력
total_values = sum(dict_bus_station.values())
print(total_values)

# 최소값 확인
min_value = min(dict_bus_station.values())
print(f"Min value: {min_value}")

# 최대값 확인
max_value = max(dict_bus_station.values())
print(f"Max value: {max_value}")

# 딕셔너리의 value 값들을 numpy array로 변환
values = np.array(list(dict_bus_station.values()))

# 중앙값(median) 계산
median = np.median(values)
print(f"Median: {median}")

# 사분위수(quantiles) 계산
lower_quartile = np.percentile(values, 25) #하위 25
upper_quartile = np.percentile(values, 75) #상위 25
print(f"Lower Quartile: {lower_quartile}")
print(f"Upper Quartile: {upper_quartile}")



In [ ]:
#folium 라이브리를 활용 지역(춘천시)의 버스 정류장 정보(정류장 위치, 정류장별 정차 노선 횟수 및 이름) 출력하기

import leafmap.foliumap as leafmap
import folium # 포리움 라이브러리
from folium.plugins import MarkerCluster

m = leafmap.Map(center=(37.87318902604981, 127.74950156136796),zoom=4) #지도의 시작값과 확대 정도를 지정하고 만들어줌
m.add_basemap("HYBRID") #위성 지도 생성을 위한 기본맵 생성
marker_cluster = MarkerCluster().add_to(m) #2. MarkerCluster에 m 추가

except_count = 0

for i in kang_bus.index : #위치번호(index)로 처음부터 끝까지 돌면서
  if kang_bus.loc[i,'위도']!='0' and kang_bus.loc[i,'경도']!='0': ### 위도와 경도가 둘 다 0이 아니면을 나타내도록
    station_name = kang_bus.loc[i,'관리번호_정류장명']
    station_stop_name = '없음'
    #print(station_name)
    ### 우리가 필요한 정보는 위도, 경도, 상호명, 주소
    lat = kang_bus.loc[i,'위도']
    lon = kang_bus.loc[i,'경도']
    title = kang_bus.loc[i,'정류장명']

    icon_color = 'black' ###색지정(파란색으로)
    try:
      if '수협' in station_name:
        print(station_name,dict_bus_station[station_name])
      if dict_bus_station[station_name] == 0: #데이터 없음
        station_stop_number = 0
        station_stop_number = '정차수 0'
      elif dict_bus_station[station_name] >= 8:#상위 25%
        icon_color = 'blue'
        station_stop_number = dict_bus_station[station_name]
        station_stop_name = dict_bus_station_name[station_name]
      elif dict_bus_station[station_name] <= 4: #하위 25%
        icon_color ='red'
        station_stop_number = dict_bus_station[station_name]
        station_stop_name = dict_bus_station_name[station_name]
      else: #25~75% 구간
        icon_color = 'green'
        station_stop_number = dict_bus_station[station_name]
        station_stop_name = dict_bus_station_name[station_name]
    except Exception as e:
      #print(f'{e} 발생')
      icon_color = 'gray' ###색지정(파란색으로)
      station_stop_number = '정류장 위치 정보에는 있지만 노선정보가 없습니다'
      except_count+=1




    folium.Marker([lat,lon], popup=folium.Popup(f"""
        <b>정류장 노선 정차 횟수:</b> {station_stop_number}<br>
        <b>정류장 정차 노선 이름:</b> {station_stop_name}""", max_width=300), tooltip=title,icon=folium.Icon(color=icon_color)).add_to(marker_cluster)
print("정류장은 있지만 노선 정보가 없는 정류장의 수 : ",except_count)
m


In [32]:
m.to_html('춘천시 버스정류장 위치 및 노선 정보 시각화.html')